In [1]:
!pip install pymysql pandas scikit-learn vaderSentiment SpeechRecognition

  Obtaining dependency information for pymysql from https://files.pythonhosted.org/packages/0c/94/e4181a1f6286f545507528c78016e00065ea913276888db2262507693ce5/PyMySQL-1.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for vaderSentiment from https://files.pythonhosted.org/packages/76/fc/310e16254683c1ed35eeb97386986d6c00bc29df17ce280aed64d55537e9/vaderSentiment-3.3.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for SpeechRecognition from https://files.pythonhosted.org/packages/e6/28/b5e6e769002e46a5edef16871884721d4b68da31dbd4509db6ea50f8b224/SpeechRecognition-3.10.4-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   --------------------------- ------------ 30.7/45.0 kB ? eta -:--:--
   --------------------------- ------------ 30.7/45.0 kB ? eta -:--:--
   --------------------------- ------------ 30.7/45.0 kB ? eta -:--:--
   ---------------------------------------- 45.0/45.0 kB 318.6 kB/s eta 0:00:

   --------------------------------- ------ 27.8/32.8 MB 114.6 kB/s eta 0:00:45
   --------------------------------- ------ 27.8/32.8 MB 115.1 kB/s eta 0:00:44
   --------------------------------- ------ 27.8/32.8 MB 115.1 kB/s eta 0:00:44
   --------------------------------- ------ 27.8/32.8 MB 116.3 kB/s eta 0:00:44
   --------------------------------- ------ 27.8/32.8 MB 116.5 kB/s eta 0:00:44
   --------------------------------- ------ 27.8/32.8 MB 116.5 kB/s eta 0:00:44
   --------------------------------- ------ 27.8/32.8 MB 116.5 kB/s eta 0:00:44
   --------------------------------- ------ 27.8/32.8 MB 116.5 kB/s eta 0:00:44
   --------------------------------- ------ 27.8/32.8 MB 116.5 kB/s eta 0:00:44
   --------------------------------- ------ 27.9/32.8 MB 117.3 kB/s eta 0:00:43
   --------------------------------- ------ 27.9/32.8 MB 117.3 kB/s eta 0:00:43
   --------------------------------- ------ 27.9/32.8 MB 118.0 kB/s eta 0:00:42
   --------------------------------- ---

In [2]:
import pymysql
import datetime
import pandas as pd
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import os
import subprocess
import speech_recognition as sr

In [3]:
sid = SentimentIntensityAnalyzer()
recognizer = sr.Recognizer()

In [4]:
db_params = {
    'host': '127.0.0.1',
    'port': 3306,
    'user': 'root',
    'password': 'root',
    'database': 'negotiate',
    'charset': 'utf8'
}

In [5]:
def view_reviews():
    output = '<table border="1" width="100%">'
    output += '<tr><th>Username</th><th>Review</th><th>Sentiment</th></tr>'
    
    with pymysql.connect(**db_params) as con:
        cur = con.cursor()
        cur.execute("SELECT * FROM reviews")
        rows = cur.fetchall()
        for row in rows:
            output += f"<tr><td>{row[0]}</td><td>{row[1]}</td><td>{row[2]}</td></tr>"
    
    return output

In [6]:
def view_orders(username):
    output = '<table border="1" width="100%">'
    output += '<tr><th>Purchaser Name</th><th>Product ID</th><th>Product Name</th><th>Amount</th><th>Purchase Date</th></tr>'
    
    with pymysql.connect(**db_params) as con:
        cur = con.cursor()
        cur.execute("SELECT * FROM purchaseorder WHERE username=%s", (username,))
        rows = cur.fetchall()
        for row in rows:
            output += f"<tr><td>{row[0]}</td><td>{row[1]}</td><td>{row[2]}</td><td>{row[3]}</td><td>{row[4]}</td></tr>"
    
    return output

In [7]:
def post_review(username, review):
    sentiment_dict = sid.polarity_scores(review)
    compound = sentiment_dict['compound']
    
    if compound >= 0.05:
        result = 'Positive'
    elif compound <= -0.05:
        result = 'Negative'
    else:
        result = 'Neutral'
    
    with pymysql.connect(**db_params) as con:
        cur = con.cursor()
        cur.execute(
            "INSERT INTO reviews(username, review, sentiment) VALUES (%s, %s, %s)",
            (username, review, result)
        )
        con.commit()
        status = "Error in taking review"
        if cur.rowcount == 1:
            status = f'Your review accepted & sentiment predicted: {result}'
    
    return status

In [8]:
def chat_data(query, predicted_price):
    output = "Sorry! I am not trained for the given question."
    
    if 'price' in query.lower():
        output = f"You can get the product at: ${predicted_price}"
    elif any(word in query.lower() for word in ["final", "discount", "my"]):
        discount = (predicted_price / 100) * 5
        predicted_price -= discount
        output = f"The final price you can get for this product is: ${predicted_price}"
    
    return output

In [10]:
username = 'test_user'  # Change as needed
print(view_orders(username))

OperationalError: (2003, "Can't connect to MySQL server on '127.0.0.1' ([WinError 10061] No connection could be made because the target machine actively refused it)")